In [2]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/dataset/Anju/fall/data0.csv").drop('timestamp', axis=1)

df.head()

,ax,ay,az,gx,gy,gz
0,-76.94,-0.24,-55.07,868,-11876,7884
1,-21.74,-53.05,-23.84,-12730,-7288,7902
2,-21.46,-52.71,-24.14,-11992,-7628,8076
3,-21.11,-52.31,-24.50,-11186,-7884,7990
4,-20.17,-51.15,-25.53,-9962,-8348,7280


In [3]:
for col in df.columns:
  print(col,min(df[col]), max(df[col]))

ax -168.99 179.39
ay -53.05 82.23
az -76.49 -3.88
gx -25064 32767
gy -11876 5016
gz -10162 24132


In [4]:
import os
import random
root = "/content/drive/MyDrive/dataset"
gen = os.walk(root)
list_of_folders = next(gen)[1]
list_of_folders.sort()

train = []
valid = []

for folder in list_of_folders:
  r1 = os.path.join(root, folder)
  g = os.walk(r1)
  
  for subfolder in next(g)[1]:
    r2 = (os.path.join(r1, subfolder))

    files = [f for f in os.listdir(r2) if os.path.isfile(os.path.join(r2, f))]

    for file_name in files:
      if random.uniform(0, 1) < 0.8:
        train.append([os.path.join(r2, file_name), 1 if subfolder == "fall" else 0])
      else:
        valid.append([os.path.join(r2, file_name), 1 if subfolder == "fall" else 0])

# 80 - 20 split to traning & validation data

In [5]:
import pandas as pd

train_df = pd.DataFrame(train, columns=['path', 'fall'])
valid_df = pd.DataFrame(valid, columns=['path', 'fall'])

In [6]:
train_df.fall.value_counts()

1    47
0    45
Name: fall, dtype: int64

In [7]:
shapes = []
import torch

for i in range(len(train_df)):
  csv_path = train_df.iloc[i, 0]
  mat = pd.read_csv(csv_path)
  mat = [list(mat[x]) for x in ['ax', 'ay', 'az', 'gx', 'gy', 'gz']]
  mat = torch.tensor(mat)
  if mat.shape[1] < 10:
    print(csv_path)
  shapes.append(mat.shape[1])

print(shapes)
mat_min = min(shapes)

[126, 120, 121, 120, 124, 121, 120, 123, 119, 122, 108, 127, 131, 128, 129, 127, 132, 138, 139, 144, 136, 140, 137, 136, 134, 134, 136, 135, 128, 130, 141, 136, 138, 132, 143, 92, 137, 137, 141, 142, 138, 137, 135, 139, 142, 121, 121, 123, 94, 120, 122, 119, 128, 126, 128, 128, 133, 131, 88, 130, 132, 133, 135, 134, 137, 134, 121, 129, 133, 132, 134, 134, 135, 132, 135, 133, 136, 127, 127, 128, 130, 129, 129, 131, 132, 128, 137, 138, 134, 137, 138, 136]


In [8]:
import os
import torch
import pandas as pd
from torch.utils.data import Dataset, IterableDataset

def mat_to_tensor(mat_path):
  mat = pd.read_csv(mat_path)
  mat = mat.dropna()
  # print(mat.info())
  mat = [list(mat[x]) for x in ['ax', 'ay', 'az', 'gx', 'gy', 'gz']]
  mat = torch.tensor(mat)
  return mat

class IMUSet(Dataset):
    def __init__(self, data_df):
        self.data_df = data_df # data_df == train_df / valid_df

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        mat_path = self.data_df.iloc[idx, 0]
        # print(mat_path)
        t = mat_to_tensor(mat_path)
        label = self.data_df.iloc[idx, 1]
        return t, label

In [9]:
from torch.utils.data import DataLoader

imuset = IMUSet(train_df) # make dataset

train_dataloader = DataLoader(imuset, shuffle=True) # load data multiple 

In [10]:
valid_dataset = IMUSet(valid_df)
valid_dataloader = DataLoader(valid_dataset, shuffle=True)

valid_features, valid_labels = next(iter(valid_dataloader))

valid_features.shape, valid_labels.shape

(torch.Size([1, 6, 134]), torch.Size([1]))

# **LSTM**

In [11]:
from torch import nn
import torch.nn.functional as F

class FallDetector(nn.Module):
    def __init__(self):
        super(FallDetector, self).__init__()
        self.lstm = nn.LSTM(6, 10, num_layers=1)
        self.hidden2tag = nn.Linear(10, 1)

    def forward(self, seq):
        output, (h_n, c_n) = self.lstm(seq.view(len(seq), 1, -1))
        tag_space = self.hidden2tag(c_n.view(1, -1))
        # print(tag_space.data)
        tag_scores = torch.sigmoid(tag_space)
        return tag_scores.view(-1)

In [12]:
model = FallDetector()

In [13]:
import torch.optim as optim
model = FallDetector()
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters()) # SGD & RMSprop & Adam : alternatives

In [14]:
import numpy as np

from sklearn.metrics import confusion_matrix, accuracy_score, recall_score

# helper function, for calculating accuracy
def get_statistics(y_true, y_pred):
    y_pred = np.concatenate(tuple(y_pred)) > 0.5
    y_true = np.concatenate(tuple([[t for t in y] for y in y_true])).reshape(y_pred.shape)
    # print("Accuracy score: ", (y_true == y_pred).sum() / float(len(y_true)))
    print("Accuracy: ", accuracy_score(y_true, y_pred))
    print("Recall: ", recall_score(y_true, y_pred))
    print(len(y_true + y_pred))
    print("Confusion Matrix")
    print(confusion_matrix(y_true, y_pred))

In [15]:

for epoch in range(50): 
    labels = []
    preds = []
    loss = []

    i = 0
    for seq, label in train_dataloader:
        model.zero_grad()

        seq = seq.reshape((-1, 6))
        tag_scores = model(seq.to(torch.float))
        preds.append(tag_scores.data)
        labels.append(label.to(torch.float).data)

        loss = loss_function(tag_scores, label.to(torch.float))
        loss.backward()
        optimizer.step()


        i += 1
    print("Epoch Done")
    get_statistics(labels, preds)
  

Epoch Done
Accuracy:  0.4782608695652174
Recall:  0.10638297872340426
92
Confusion Matrix
[[39  6]
 [42  5]]
Epoch Done
Accuracy:  0.6195652173913043
Recall:  0.3829787234042553
92
Confusion Matrix
[[39  6]
 [29 18]]
Epoch Done
Accuracy:  0.7282608695652174
Recall:  0.6595744680851063
92
Confusion Matrix
[[36  9]
 [16 31]]
Epoch Done
Accuracy:  0.6956521739130435
Recall:  0.6382978723404256
92
Confusion Matrix
[[34 11]
 [17 30]]
Epoch Done
Accuracy:  0.7608695652173914
Recall:  0.6382978723404256
92
Confusion Matrix
[[40  5]
 [17 30]]
Epoch Done
Accuracy:  0.75
Recall:  0.5957446808510638
92
Confusion Matrix
[[41  4]
 [19 28]]
Epoch Done
Accuracy:  0.7391304347826086
Recall:  0.5957446808510638
92
Confusion Matrix
[[40  5]
 [19 28]]
Epoch Done
Accuracy:  0.75
Recall:  0.7021276595744681
92
Confusion Matrix
[[36  9]
 [14 33]]
Epoch Done
Accuracy:  0.75
Recall:  0.6808510638297872
92
Confusion Matrix
[[37  8]
 [15 32]]
Epoch Done
Accuracy:  0.7391304347826086
Recall:  0.6808510638297872


In [16]:

labels = []
preds = []
loss = []
model.eval()

for seq, label in valid_dataloader:

    seq = seq.reshape((-1, 6))
    tag_scores = model(seq.to(torch.float))
    preds.append(tag_scores.data)
    labels.append(label.to(torch.float).data)

print("Epoch Done")
get_statistics(labels, preds)

Epoch Done
Accuracy:  0.8846153846153846
Recall:  0.9166666666666666
26
Confusion Matrix
[[12  2]
 [ 1 11]]


In [17]:
torch.save(model.state_dict(), 'model_weights.pt')

In [21]:
from torch import nn
import torch.nn.functional as F

class FallDetector(nn.Module):
    def __init__(self):
        super(FallDetector, self).__init__()
        self.lstm = nn.LSTM(6, 10, num_layers=1)
        self.hidden2tag = nn.Linear(10, 1)

    def forward(self, seq):
        output, (h_n, c_n) = self.lstm(seq.view(len(seq), 1, -1))
        tag_space = self.hidden2tag(c_n.view(1, -1))
        # print(tag_space.data)
        tag_scores = torch.sigmoid(tag_space) # output changed to 0-1
        return tag_scores.view(-1)

model = FallDetector()
model.load_state_dict(torch.load('model_weights.pt'))
model.eval()

FallDetector(
  (lstm): LSTM(6, 10)
  (hidden2tag): Linear(in_features=10, out_features=1, bias=True)
)

In [24]:
import tensorflow as tf

In [28]:
!pip install onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
import onnx

In [34]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.5 MB/s eta 0:00:00


In [35]:
!git clone https://github.com/onnx/onnx-tensorflow.git && cd onnx-tensorflow && pip install -e .

Cloning into 'onnx-tensorflow'...
remote: Enumerating objects: 6516, done.
remote: Counting objects: 100% (465/465), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 6516 (delta 322), reused 394 (delta 273), pack-reused 6051
Receiving objects: 100% (6516/6516), 1.98 MiB | 3.52 MiB/s, done.
Resolving deltas: 100% (5047/5047), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/onnx-tensorflow
  Preparing metadata (setup.py) ... done
  Running setup.py develop for onnx-tf


In [2]:
!pip install torch-vision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, IterableDataset
import tensorflow as tf

import onnx
from onnx_tf.backend import prepare

Load the saved pytorch model and export it as an ONNX file

In [5]:


class FallDetector(nn.Module):
    def __init__(self):
        super(FallDetector, self).__init__()
        self.lstm = nn.LSTM(6, 10, num_layers=1)
        self.hidden2tag = nn.Linear(10, 1)

    def forward(self, seq):
        output, (h_n, c_n) = self.lstm(seq.view(len(seq), 1, -1))
        tag_space = self.hidden2tag(c_n.view(1, -1))
        # print(tag_space.data)
        tag_scores = torch.sigmoid(tag_space) # output changed to 0-1
        return tag_scores.view(-1)

model = FallDetector()
model.load_state_dict(torch.load('model_weights.pt'))
model.eval()

FallDetector(
  (lstm): LSTM(6, 10)
  (hidden2tag): Linear(in_features=10, out_features=1, bias=True)
)

In [8]:
queue = []
for i in range(20):
  dummy_input = [0] * 6
  queue.append(dummy_input)

dummy_input = torch.tensor(queue, dtype=torch.float)
print(dummy_input.shape)

torch.onnx.export(model, dummy_input, "model.onnx")

torch.Size([20, 6])


<ipython-input-5-77ccf8d4e93d>:8: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  output, (h_n, c_n) = self.lstm(seq.view(len(seq), 1, -1))
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:209: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.input_size != input.size(-1):
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:230: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not general

Load the ONNX file and import it into tensorflow

In [10]:
# Load the onnx file
onnx_model = onnx.load('model.onnx')

# Import onnx model to tensorflow
tf_rep = prepare(onnx_model)

Run and Test Tensorflow model

In [14]:
queue = []
for i in range(20):
  dummy_input = [0] * 6
  queue.append(dummy_input)

dummy_input = torch.tensor(queue, dtype=torch.float)

output = tf_rep.run(dummy_input)
print(output)

Outputs(_0=array([0.04108686], dtype=float32))


In [15]:
tf_rep.export_graph('tf_model.pb')

/usr/local/lib/python3.8/dist-packages/keras/legacy_tf_layers/base.py:627: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  self.updates, tf.compat.v1.GraphKeys.UPDATE_OPS
/usr/local/lib/python3.8/dist-packages/keras/legacy_tf_layers/base.py:627: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  self.updates, tf.compat.v1.GraphKeys.UPDATE_OPS
